# Metrics Correlation Analysis: GCM-RCM Error Metrics

**Objective**: Analyze correlations between error metrics computed for climate model (GCM-RCM) pairs.

**Data Structure**: For each combination of `(region, gridpoint, physical_variable, model)`, we have multiple error metrics (ACC, d, KGE, etc.). We compute correlations between these metrics to identify redundancy and orthogonal error facets.

**Output**: Scatter plot matrix (pairplot), correlation heatmaps, and statistical summaries.


## 1. Project Setup & Configuration

This section defines the structure and user-configurable parameters for the analysis.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr, spearmanr, kendalltau

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

import os
OUTPUT_DIR = "./output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("✓ Libraries imported successfully")


✓ Libraries imported successfully


### Configuration Parameters

**Programmer**: Edit the following parameters to customize your analysis.

In [3]:
# Available regions. Set to None or empty list to include ALL regions. Analyze only France region: ['FR']
REGIONS = ['FR']  

# Physical variables: 'ppt' (precipitation) or 'tas' (temperature). Set to None to include both. Only precipitation: ['ppt']
PHYSICAL_VARIABLES = ['ppt']

# Metric abbreviations to include in the correlation analysis. None = all metrics; or specify like ['ACC', 'd', 'KGE (2009)', 'BM']
METRIC_ABBREVIATIONS = ['H10 (MAHE)', 'd', 'dr', 'NED', 'MV', 'KGE (2009)']

# Correlation method: 'pearson', 'spearman', or 'kendall'
CORRELATION_METHOD = 'pearson'

print(f"Regions: {REGIONS if REGIONS else 'ALL'}")
print(f"Physical Variables: {PHYSICAL_VARIABLES if PHYSICAL_VARIABLES else 'ALL'}")
print(f"Metric Abbreviations: {METRIC_ABBREVIATIONS if METRIC_ABBREVIATIONS else 'ALL'}")
print(f"Correlation Method: {CORRELATION_METHOD}")


Regions: ['FR']
Physical Variables: ['ppt']
Metric Abbreviations: ['H10 (MAHE)', 'd', 'dr', 'NED', 'MV', 'KGE (2009)']
Correlation Method: pearson


## 2. Load Data from Database

We'll query the error and metrics tables from the database (as described in the SQL schema).

In [4]:
import sys
sys.path.append('..') 

from utils import get_db_connection

try:
    engine = get_db_connection()
    conn = engine.connect()
    print(f"✓ Connected to PostgreSQL database")
except Exception as e:
    print(f"✗ Error connecting to database: {e}")
    print("  Make sure DB_NAME, DB_USER, DB_PASSWORD, DB_HOST, DB_PORT environment variables are set")


✓ Connected to PostgreSQL database


In [6]:
where_clauses = []

if REGIONS is not None and len(REGIONS) > 0:
    regions_str = "', '".join(REGIONS)
    where_clauses.append(f"error.region IN ('{regions_str}')")

if PHYSICAL_VARIABLES is not None and len(PHYSICAL_VARIABLES) > 0:
    vars_str = "', '".join(PHYSICAL_VARIABLES)
    where_clauses.append(f"error.physical_variable IN ('{vars_str}')")

if METRIC_ABBREVIATIONS is not None and len(METRIC_ABBREVIATIONS) > 0:
    metrics_str = "', '".join(METRIC_ABBREVIATIONS)
    where_clauses.append(f"metrics.metric_name IN ('{metrics_str}')")

where_clause = " AND ".join(where_clauses)
if where_clause:
    where_clause = f"WHERE {where_clause}"

query = f"""
SELECT 
    error.region,
    error.gridpoint,
    error.physical_variable,
    error.model,
    error.rcm_id,
    error.gcm_id,
    error.metric_id,
    metrics.metric_name,
    error.mat_vector
FROM 
    error 
LEFT JOIN 
    metrics ON metrics.id = error.metric_id
{where_clause}
"""

df_raw = pd.read_sql_query(query, conn)
print(f"✓ Data loaded: {len(df_raw)} rows")
print(f"  Columns: {list(df_raw.columns)}")
print(f"\nFirst few rows:")
print(df_raw.head(5))


✓ Data loaded: 329478 rows
  Columns: ['region', 'gridpoint', 'physical_variable', 'model', 'rcm_id', 'gcm_id', 'metric_id', 'metric_name', 'mat_vector']

First few rows:
  region  gridpoint physical_variable  model  rcm_id  gcm_id  metric_id  \
0     FR          2               ppt     18       1       3         95   
1     FR          2               ppt     19       2       3         95   
2     FR          2               ppt     40       6       4         95   
3     FR          2               ppt     35       2       3         95   
4     FR          2               ppt     46      11       5         95   

  metric_name  mat_vector  
0          dr    0.354677  
1          dr    0.343277  
2          dr    0.351538  
3          dr    0.383483  
4          dr    0.293005  


## 3. Data Filtering & Preprocessing

Apply user-defined filters to select the region(s), physical variable(s), and metric(s) of interest.

In [7]:
df = df_raw.copy()

print(f"✓ Data retrieved (pre-filtered by database query):")
print(f"  Total rows: {len(df)}")
print(f"  Regions: {sorted(df['region'].unique())}")
print(f"  Physical variables: {sorted(df['physical_variable'].unique())}")
print(f"  Unique metrics: {df['metric_name'].nunique()}")

print(f"\nMissing values:")
print(df.isnull().sum())

print(f"\nDataset shape: {df.shape}")
print(f"Metrics: {sorted(df['metric_name'].unique())}")
print(f"Unique models: {df['model'].nunique()}")
print(f"Unique gridpoints: {df['gridpoint'].nunique()}")


✓ Data retrieved (pre-filtered by database query):
  Total rows: 329478
  Regions: ['FR']
  Physical variables: ['ppt']
  Unique metrics: 6

Missing values:
region               0
gridpoint            0
physical_variable    0
model                0
rcm_id               0
gcm_id               0
metric_id            0
metric_name          0
mat_vector           0
dtype: int64

Dataset shape: (329478, 9)
Metrics: ['H10 (MAHE)', 'KGE (2009)', 'MV', 'NED', 'd', 'dr']
Unique models: 89
Unique gridpoints: 617


## 4. Create Pivot Table for Correlation Analysis

Transform the data so that each row represents a unique `(region, gridpoint, physical_variable, model)` grouping,  
and each column is a metric. This structure allows us to compute correlations between metrics.

In [8]:
# Create a grouping key for (region, gridpoint, physical_variable, model)
df['group_key'] = df.apply(
    lambda row: f"{row['region']}_{row['gridpoint']}_{row['physical_variable']}_{row['model']}", 
    axis=1
)

# Pivot: rows are group_keys, columns are metric names, values are error values
pivot_df = df.pivot_table(
    index='group_key',
    columns='metric_name',
    values='mat_vector',
    aggfunc='first'  # Should be unique per group_key, but just in case
)

print(f"✓ Pivot table created:")
print(f"  Shape: {pivot_df.shape}")
print(f"  Rows (unique group_keys): {pivot_df.shape[0]}")
print(f"  Columns (metrics): {pivot_df.shape[1]}")
print(f"\nMetrics in pivot table:")
print(pivot_df.columns.tolist())
print(f"\nFirst few rows:")
print(pivot_df.head())

print(f"\nMissing values per metric:")
print(pivot_df.isnull().sum())


✓ Pivot table created:
  Shape: (54913, 6)
  Rows (unique group_keys): 54913
  Columns (metrics): 6

Metrics in pivot table:
['H10 (MAHE)', 'KGE (2009)', 'MV', 'NED', 'd', 'dr']

First few rows:
metric_name    H10 (MAHE)  KGE (2009)        MV        NED         d        dr
group_key                                                                     
FR_100_ppt_1     1.242379   -0.092011  2.478668  21.851604  0.379031  0.371351
FR_100_ppt_10    0.935961   -0.012109  1.197039  18.082803  0.415196  0.213969
FR_100_ppt_11    0.861331    0.050020  1.104480  17.909459  0.435181  0.286322
FR_100_ppt_12    0.849339    0.092962  1.096309  17.605076  0.447742  0.339210
FR_100_ppt_13    0.942766    0.032100  1.449987  19.549915  0.413416  0.239227

Missing values per metric:
metric_name
H10 (MAHE)    0
KGE (2009)    0
MV            0
NED           0
d             0
dr            0
dtype: int64


In [9]:
pivot_df_clean = pivot_df.dropna()

print(f"✓ Removed rows with missing values:")
print(f"  Original: {pivot_df.shape[0]} rows")
print(f"  After dropna: {pivot_df_clean.shape[0]} rows")
print(f"  Rows removed: {pivot_df.shape[0] - pivot_df_clean.shape[0]}")


✓ Removed rows with missing values:
  Original: 54913 rows
  After dropna: 54913 rows
  Rows removed: 0


## 5. Compute Pairwise Correlations

Calculate correlation coefficients between all pairs of metrics.

In [10]:
corr_matrix = pivot_df_clean.corr(method=CORRELATION_METHOD)

print(f"✓ Correlation matrix computed ({CORRELATION_METHOD}):")
print(f"  Shape: {corr_matrix.shape}")
print(f"\nCorrelation Matrix:")
print(corr_matrix)

✓ Correlation matrix computed (pearson):
  Shape: (6, 6)

Correlation Matrix:
metric_name  H10 (MAHE)  KGE (2009)        MV       NED         d        dr
metric_name                                                                
H10 (MAHE)     1.000000   -0.206456  0.796779  0.735813  0.032381 -0.224065
KGE (2009)    -0.206456    1.000000  0.301979  0.207025  0.817592  0.875730
MV             0.796779    0.301979  1.000000  0.898409  0.324015  0.334772
NED            0.735813    0.207025  0.898409  1.000000  0.172221  0.316493
d              0.032381    0.817592  0.324015  0.172221  1.000000  0.600129
dr            -0.224065    0.875730  0.334772  0.316493  0.600129  1.000000


In [11]:
def pairwise_corr_with_pvalues(df, method='pearson'):
    """Compute pairwise correlations and p-values."""
    cols = df.columns
    n_cols = len(cols)
    results = []
    
    for i in range(n_cols):
        for j in range(i + 1, n_cols):
            metric1 = cols[i]
            metric2 = cols[j]
            
            if method == 'pearson':
                corr, pval = pearsonr(df[metric1], df[metric2])
            elif method == 'spearman':
                corr, pval = spearmanr(df[metric1], df[metric2])
            elif method == 'kendall':
                corr, pval = kendalltau(df[metric1], df[metric2])
            else:
                raise ValueError(f"Unknown method: {method}")
            
            results.append({
                'Metric 1': metric1,
                'Metric 2': metric2,
                'Correlation': corr,
                'P-value': pval,
                'N': len(df)
            })
    
    return pd.DataFrame(results)

In [12]:
pairwise_df = pairwise_corr_with_pvalues(pivot_df_clean, method=CORRELATION_METHOD)

pairwise_df['Abs_Corr'] = pairwise_df['Correlation'].abs()
pairwise_df = pairwise_df.sort_values('Abs_Corr', ascending=False).drop('Abs_Corr', axis=1)

print(f"\n✓ Pairwise correlations ({len(pairwise_df)} pairs):")
print(pairwise_df.to_string(index=False))


✓ Pairwise correlations (15 pairs):
  Metric 1   Metric 2  Correlation      P-value     N
        MV        NED     0.898409 0.000000e+00 54913
KGE (2009)         dr     0.875730 0.000000e+00 54913
KGE (2009)          d     0.817592 0.000000e+00 54913
H10 (MAHE)         MV     0.796779 0.000000e+00 54913
H10 (MAHE)        NED     0.735813 0.000000e+00 54913
         d         dr     0.600129 0.000000e+00 54913
        MV         dr     0.334772 0.000000e+00 54913
        MV          d     0.324015 0.000000e+00 54913
       NED         dr     0.316493 0.000000e+00 54913
KGE (2009)         MV     0.301979 0.000000e+00 54913
H10 (MAHE)         dr    -0.224065 0.000000e+00 54913
KGE (2009)        NED     0.207025 0.000000e+00 54913
H10 (MAHE) KGE (2009)    -0.206456 0.000000e+00 54913
       NED          d     0.172221 0.000000e+00 54913
H10 (MAHE)          d     0.032381 3.201464e-14 54913


- $H_0$: zero correlation  
- $H_1$: non-zero correlation

This **p-value** is the two-sided probability of observing a correlation at least as extreme as the measured one under the null hypothesis.

Assumptions
- **Pearson**: assumes a linear relationship and (approximately) bivariate normality.  
- **Spearman** / **Kendall**: rank-based (nonparametric), less sensitive to outliers or non-normality.

Can also be seen as association:
- $H_0$: The population correlation is zero (no association)
- $H_1$: The population correlation is non-zero (association exists)

## 6. Scatter Plot Matrix (Pairplot)

Generate a pairplot to visualize relationships between metrics (similar to the attached image).

In [ ]:
# Create pairplot
plt.figure(figsize=(16, 16))
g = sns.pairplot(
    pivot_df_clean,
    diag_kind='hist',
    plot_kws={'alpha': 0.6, 's': 30},
    diag_kws={'bins': 30, 'edgecolor': 'k', 'alpha': 0.7}
)

# Add correlation coefficients to upper triangle
def add_correlation_annotations(g, corr_matrix):
    """Add correlation values to upper triangle of pairplot."""
    for i, ax_row in enumerate(g.axes):
        for j, ax in enumerate(ax_row):
            if i < j:  # Upper triangle
                col_i = g.data.columns[i]
                col_j = g.data.columns[j]
                corr_val = corr_matrix.loc[col_i, col_j]
                ax.text(
                    0.5, 0.5,
                    f'{corr_val:.2f}***' if abs(corr_val) > 0.7 else 
                    f'{corr_val:.2f}**' if abs(corr_val) > 0.5 else
                    f'{corr_val:.2f}',
                    transform=ax.transAxes,
                    ha='center', va='center',
                    fontsize=12, fontweight='bold',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8)
                )
                ax.set_title('')

add_correlation_annotations(g, corr_matrix)

plt.suptitle(
    f'Metrics Correlation Analysis - Scatter Plot Matrix\n'
    f'Region(s): {REGIONS if REGIONS else "ALL"} | '
    f'Variable(s): {PHYSICAL_VARIABLES if PHYSICAL_VARIABLES else "ALL"} | '
    f'Method: {CORRELATION_METHOD}',
    fontsize=14, y=1.00
)
plt.tight_layout()

# Save the figure
output_file = f"{OUTPUT_DIR}/pairplot_{CORRELATION_METHOD}.png"
plt.savefig(output_file, dpi=300, bbox_inches='tight')
print(f"✓ Pairplot saved to: {output_file}")
plt.show()


## 7. Correlation Heatmap

Visualize the correlation matrix as a heatmap with hierarchical clustering.

In [ ]:
# Create heatmap with hierarchical clustering
plt.figure(figsize=(12, 10))
sns.clustermap(
    corr_matrix,
    cmap='RdBu_r',
    center=0,
    vmin=-1, vmax=1,
    annot=True,
    fmt='.2f',
    cbar_kws={'label': f'{CORRELATION_METHOD.capitalize()} Correlation'},
    linewidths=0.5,
    linecolor='gray'
)

plt.suptitle(
    f'Hierarchical Correlation Heatmap ({CORRELATION_METHOD})\n'
    f'Region(s): {REGIONS if REGIONS else "ALL"} | Variable(s): {PHYSICAL_VARIABLES if PHYSICAL_VARIABLES else "ALL"}',
    fontsize=14, y=0.98
)
plt.tight_layout()

output_file = f"{OUTPUT_DIR}/heatmap_clustered_{CORRELATION_METHOD}.png"
plt.savefig(output_file, dpi=300, bbox_inches='tight')
print(f"✓ Heatmap saved to: {output_file}")
plt.show()


## 8. Export Results

Save correlation matrices and pairwise correlations to CSV files.

In [ ]:
# Save correlation matrix to CSV
corr_matrix.to_csv(f"{OUTPUT_DIR}/correlation_matrix_{CORRELATION_METHOD}.csv")
print(f"✓ Correlation matrix saved to: {OUTPUT_DIR}/correlation_matrix_{CORRELATION_METHOD}.csv")

# Save pairwise correlations to CSV
pairwise_df.to_csv(f"{OUTPUT_DIR}/pairwise_correlations_{CORRELATION_METHOD}.csv", index=False)
print(f"✓ Pairwise correlations saved to: {OUTPUT_DIR}/pairwise_correlations_{CORRELATION_METHOD}.csv")

# Save summary statistics
summary_stats = pd.DataFrame({
    'Metric': corr_matrix.columns,
    'Mean': pivot_df_clean.mean(),
    'Std': pivot_df_clean.std(),
    'Min': pivot_df_clean.min(),
    'Max': pivot_df_clean.max(),
})
summary_stats.to_csv(f"{OUTPUT_DIR}/metrics_summary_statistics.csv", index=False)
print(f"✓ Summary statistics saved to: {OUTPUT_DIR}/metrics_summary_statistics.csv")

# Save analysis metadata
metadata = {
    'Analysis Date': pd.Timestamp.now(),
    'Correlation Method': CORRELATION_METHOD,
    'Regions': str(REGIONS),
    'Physical Variables': str(PHYSICAL_VARIABLES),
    'Metric Abbreviations': str(METRIC_ABBREVIATIONS),
    'Number of Rows': pivot_df_clean.shape[0],
    'Number of Metrics': pivot_df_clean.shape[1],
    'Metrics': ', '.join(pivot_df_clean.columns)
}

metadata_df = pd.DataFrame(list(metadata.items()), columns=['Parameter', 'Value'])
metadata_df.to_csv(f"{OUTPUT_DIR}/analysis_metadata.csv", index=False)
print(f"✓ Analysis metadata saved to: {OUTPUT_DIR}/analysis_metadata.csv")

print(f"\n✓ All results exported to: {OUTPUT_DIR}/")


## 9. Summary & Insights

Summary of the correlation analysis results.

In [ ]:
print("=" * 70)
print("ANALYSIS SUMMARY")
print("=" * 70)
print(f"\nData Summary:")
print(f"  Total rows analyzed: {pivot_df_clean.shape[0]}")
print(f"  Number of metrics: {pivot_df_clean.shape[1]}")
print(f"  Correlation method: {CORRELATION_METHOD}")

print(f"\nMetrics included:")
for i, metric in enumerate(pivot_df_clean.columns, 1):
    print(f"  {i}. {metric}")

print(f"\nTop 10 Strongest Correlations (by absolute value):")
top_corr = pairwise_df.head(10)
for idx, row in top_corr.iterrows():
    sig_marker = "***" if row['P-value'] < 0.001 else "**" if row['P-value'] < 0.01 else "*" if row['P-value'] < 0.05 else "ns"
    print(f"  {row['Metric 1']:15} <-> {row['Metric 2']:15} : {row['Correlation']:7.3f} {sig_marker}")

print(f"\nTop 10 Weakest Correlations (by absolute value):")
weak_corr = pairwise_df.tail(10)
for idx, row in weak_corr.iterrows():
    sig_marker = "***" if row['P-value'] < 0.001 else "**" if row['P-value'] < 0.01 else "*" if row['P-value'] < 0.05 else "ns"
    print(f"  {row['Metric 1']:15} <-> {row['Metric 2']:15} : {row['Correlation']:7.3f} {sig_marker}")

print(f"\nSignificance levels (α = 0.05):")
sig_count = (pairwise_df['P-value'] < 0.05).sum()
print(f"  Significant correlations: {sig_count} / {len(pairwise_df)} ({100*sig_count/len(pairwise_df):.1f}%)")

print("\nNote: *** p<0.001, ** p<0.01, * p<0.05, ns = not significant")
print("=" * 70)
